In [9]:
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, ELU, Flatten, MaxPool2D
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import lecun_uniform
from tensorflow.keras.optimizers import Adam
import numpy as np

## Dataset


In [10]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")
X_train_valid = X_train_valid[:, :22, :]
X_test = X_test[:, :22, :]
y_train_valid = to_categorical(y_train_valid - 769)
y_test = to_categorical(y_test - 769)

## Shape of Data

In [11]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115, 4)
Test target shape: (443, 4)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


## Model

In [12]:
def cnn_model(time_period=1000):
    model = Sequential()

    model.add(Conv2D(16, kernel_size=(1, 10), strides=1, padding='valid', kernel_regularizer=L1L2(l1=0, l2=0.01), input_shape=(22, time_period, 1), data_format='channels_last'))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D((1, 4)))
    model.add(Dropout(0.2))

    model.add(Conv2D(32, kernel_size=(1, 10), strides=1, padding='valid', kernel_regularizer=L1L2(l1=0, l2=0.01)))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D((1, 4)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(1, 10), strides=1, padding='valid', kernel_regularizer=L1L2(l1=0, l2=0.01)))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D((1, 4)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(21, 1), strides=1, padding='valid', kernel_regularizer=L1L2(l1=0, l2=0.01)))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D((1, 4)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(4, kernel_regularizer=L1L2(l1=0, l2=0.01), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, epsilon=1e-8, decay=0.01), metrics=[categorical_accuracy])
    
    return model

## Training

In [21]:
def train(time_period=1000):
    X_train_valid_exp = np.expand_dims(X_train_valid, axis=3)
    X_test_exp = np.expand_dims(X_test, axis=3)
    lecun = lecun_uniform(seed=42)

    model = cnn_model(time_period)
    model.fit(X_train_valid_exp, y_train_valid, epochs=40, batch_size=32, validation_data=(X_test_exp, y_test), shuffle=True, verbose=1)
    train_score = model.evaluate(X_train_valid_exp, y_train_valid)
    test_score = model.evaluate(X_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    print(model.summary())
    
    return train_score, test_score


In [22]:
train_score, test_score = train()

Epoch 1/40
67/67 [==============================] - 57s 836ms/step - loss: 3.9165 - categorical_accuracy: 0.2500 - val_loss: 3.1352 - val_categorical_accuracy: 0.2460
Epoch 2/40
67/67 [==============================] - 54s 813ms/step - loss: 2.9127 - categorical_accuracy: 0.2503 - val_loss: 2.5620 - val_categorical_accuracy: 0.3476
Epoch 3/40
67/67 [==============================] - 55s 814ms/step - loss: 2.6184 - categorical_accuracy: 0.3020 - val_loss: 2.3538 - val_categorical_accuracy: 0.3657
Epoch 4/40
67/67 [==============================] - 55s 816ms/step - loss: 2.4699 - categorical_accuracy: 0.3327 - val_loss: 2.3028 - val_categorical_accuracy: 0.3409
Epoch 5/40
67/67 [==============================] - 55s 815ms/step - loss: 2.3147 - categorical_accuracy: 0.3701 - val_loss: 2.1452 - val_categorical_accuracy: 0.4221
Epoch 6/40
67/67 [==============================] - 56s 839ms/step - loss: 2.1121 - categorical_accuracy: 0.4269 - val_loss: 2.0754 - val_categorical_accuracy: 0.440

ValueError: in user code:

    /Users/billwu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /Users/billwu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/billwu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/billwu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/billwu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/billwu/opt/anaconda3/lib/python